In [1]:
import numpy as np
import pandas as pd

In [2]:
df= pd.read_csv('../TextFiles/smsspamcollection.tsv' , sep = '\t')

In [3]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [4]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [5]:
df['label'].value_counts()   # ballance

ham     4825
spam     747
Name: label, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X = df['message']
Y = df['label']


/Users/harshitsingh/opt/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [7]:
X_train, X_test, Y_train , Y_test = train_test_split(X, Y, test_size = 0.33, random_state=42)

### vECTorization 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
count_vect = CountVectorizer()

In [10]:
# FIT VECTORIZER TO THE DATA (build a vocab, count the number of words...)
# count_vect.fit(X_train)
# X_ train_counts = count_vect. transform(X_ train)
# TRANSFORM THE ORIGINAL TEXT MESSAGE-› VECTOR
X_train_counts = count_vect.fit_transform(X_train)

In [11]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [12]:
X_train.shape

(3733,)

In [13]:
X_train_counts.shape

(3733, 7082)

#### TF IDF

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer

In [15]:
tfidf_transformer = TfidfTransformer()

In [16]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [17]:
X_train_tfidf.shape

(3733, 7082)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer() # instance of vectorizer 

In [19]:
X_train_tfidf = vectorizer.fit_transform(X_train) # 

##### Train a Classifier 

In [20]:
from sklearn.svm import LinearSVC

In [21]:
clf = LinearSVC()  # instance 

In [22]:
clf.fit(X_train_tfidf, Y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [23]:
from sklearn.pipeline import Pipeline

In [27]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])# List of tuples  -> All steps in single step

In [28]:
text_clf.fit(X_train,Y_train)

/Users/harshitsingh/opt/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/utils/validation.py:209: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':


Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [29]:
X_test

3245    Squeeeeeze!! This is christmas hug.. If u lik ...
944     And also I've sorta blown him off a couple tim...
1044    Mmm thats better now i got a roast down me! i...
2484        Mm have some kanji dont eat anything heavy ok
812     So there's a ring that comes with the guys cos...
2973    Sary just need Tim in the bollox &it hurt him ...
2991    Love isn't a decision, it's a feeling. If we c...
2942    My supervisor find 4 me one lor i thk his stud...
230                    Dear good morning now only i am up
1181                           I'm in chennai velachery:)
1912    Lol grr my mom is taking forever with my presc...
1992    No other Valentines huh? The proof is on your ...
5435                    I'm wif him now buying tix lar...
4805    Er, hello, things didn‘t quite go to plan – is...
401     FREE RINGTONE text FIRST to 87131 for a poly o...
1859                     Sir, i am waiting for your call.
1344    Crazy ar he's married. Ü like gd looking guys ...
2952          

In [30]:
predictions = text_clf.predict(X_test)

In [31]:
from sklearn.metrics import confusion_matrix,classification_report

In [32]:
print(confusion_matrix(Y_test,predictions))

[[1586    7]
 [  12  234]]


In [33]:
print(classification_report(Y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

   micro avg       0.99      0.99      0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [34]:
from sklearn import metrics

In [35]:
metrics.accuracy_score(Y_test,predictions)

0.989668297988037

#### CHECK

In [36]:
text_clf.predict(["Hi how are. you doing"])

array(['ham'], dtype=object)

In [37]:
text_clf.predict(["Congratulations! you have been selected as a winner"])

array(['spam'], dtype=object)